# Constructing linear model for OER adsorption energies
---


### Import Modules

In [22]:
import os
print(os.getcwd())
import sys
import time; ti = time.time()

import pickle

import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)

import plotly.graph_objects as go

# #########################################################
from methods import (
    get_df_features_targets,
    get_df_features_targets_seoin,
    )

from methods_models import ModelAgent, GP_Regression

from proj_data import adsorbates
from proj_data import layout_shared
from proj_data import scatter_marker_props

/mnt/c/Users/RaulA/Dropbox/Computing/00_git_repos/PROJ_IrOx_OER/workflow/model_building/gaussian_process/my_data/same_features_as_seoin


In [23]:
from methods import isnotebook    
isnotebook_i = isnotebook()
if isnotebook_i:
    from tqdm.notebook import tqdm
    verbose = True
    show_plot = True
else:
    from tqdm import tqdm
    verbose = False
    show_plot = False

In [24]:
root_dir = os.path.join(
    os.environ["PROJ_irox_oer"],
    "workflow/model_building/gaussian_process/my_data/same_features_as_seoin")

### Script Inputs

In [25]:
target_ads_i = "oh"

feature_ads_i = "o"

use_seoin_data = False

if use_seoin_data:
    feature_ads_i = "o"

In [26]:
quick_easy_settings = False
if quick_easy_settings:
    k_fold_partition_size = 170
    do_every_nth_pca_comp = 4
else:
    k_fold_partition_size = 30
    do_every_nth_pca_comp = 1

In [27]:
# # TEMP
# print(111 * "TEMP | ")
# do_every_nth_pca_comp = 9

### Read Data

In [28]:
# #########################################################
df_features_targets = get_df_features_targets()

# #########################################################
df_seoin = get_df_features_targets_seoin()

### Combine mine and Seoin's data

In [29]:
if use_seoin_data:
    # Replace multiindex with index of tuples so that my data and Seoin's data can be combined
    indices = df_features_targets.index.tolist()
    df_features_targets.index = indices

    indices = df_seoin.index.tolist()
    df_seoin.index = indices

    # Remove columns that aren't shared by my and Seoin's data
    cols_0 =df_features_targets.columns.tolist()
    cols_1 = df_seoin.columns.tolist()

    cols_comb = cols_0 + cols_1

    cols_comb_unique = []
    for col_i in cols_comb:
        if col_i not in cols_comb_unique:
            cols_comb_unique.append(col_i)

    shared_cols = []
    for col_i in cols_comb_unique:
        if col_i in df_features_targets.columns and col_i in df_seoin.columns:
            shared_cols.append(col_i)

    # Combine data
    df_data = pd.concat([
        df_features_targets[shared_cols],
        df_seoin[shared_cols],
        ], axis=0)
else:
    df_data = df_features_targets

### Choosing feature columns

In [30]:
df_data = df_data[[

    # ('targets', 'g_o', ''),
    ('targets', 'g_oh', ''),

    ('data', 'stoich', ''),
    ('data', 'job_id_o', ''),
    ('data', 'job_id_oh', ''),
    ('data', 'job_id_bare', ''),

    # ('features', 'o', 'O_magmom'),
    # ('features', 'o', 'Ir_magmom'),
    # ('features', 'o', 'Ir*O_bader'),
    # ('features', 'o', 'Ir_bader'),
    # ('features', 'o', 'O_bader'),
    # ('features', 'o', 'p_band_center'),
    # ('features', 'o', 'Ir*O_bader/ir_o_mean'),

    ('features', 'o', 'active_o_metal_dist'),
    # ('features', 'o', 'angle_O_Ir_surf_norm'),
    # ('features', 'o', 'ir_o_mean'),
    ('features', 'o', 'ir_o_std'),
    # ('features', 'o', 'octa_vol'),

    ('features', 'dH_bulk', ''),
    ('features', 'volume_pa', ''),
    ('features', 'bulk_oxid_state', ''),
    ('features', 'effective_ox_state', ''),

    ]]

In [31]:
df_data

targets   data                            \
                                     g_oh stoich     job_id_o    job_id_oh   
                                                                             
compenv slab_id     active_site                                              
nersc   buvivore_13 38.0         0.145745    AB2  fidivuwi_89  nipidida_98   
        dakoputu_58 74.0         1.355680    AB2  habamehi_10  buvawasa_30   
                    75.0         1.294538    AB2  hatabufi_70  kutabadu_95   
                    76.0         1.452632    AB2  ditogotu_52  vokuvige_58   
                    77.0         0.888724    AB2  timeviri_63  madopana_89   
...                                   ...    ...          ...          ...   
slac    wonataro_02 64.0         1.316938    AB3  robivufe_82  bibupufo_15   
        wowukoli_59 27.0         0.613686    AB2  kosufovo_07  bokobitu_09   
        wufulafe_03 56.0         0.125160    AB2  wihatori_14  vibifulo_23   
                    57.0         0.418003    AB2  linovote_47  sebibiko_55   
                    58.0         0.574642    AB2  menamaka_12  sudubuwe_77   

                                                        features            \
                                 job_id_bare                   o             
                                             active_o_metal_dist  ir_o_std   
compenv slab_id     active_site                                              
nersc   buvivore_13 38.0         halavamu_98            1.782397  0.104375   
        dakoputu_58 74.0         dibomehi_65            1.750519  0.219401   
                    75.0         sanegusa_24            1.761406  0.163273   
                    76.0         metuwifa_55            1.744237  0.211677   
                    77.0         dunopafu_19            1.761731  0.214784   
...                                      ...                 ...       ...   
slac    wonataro_02 64.0         bikoriko_02            1.764480  0.114769   
        wowukoli_59 27.0         wabiwuti_62            1.753292  0.238801   
        wufulafe_03 56.0         hibikofe_60            1.766557  0.102071   
                    57.0         vehehomi_38            1.780165  0.092489   
                    58.0         vunosepi_77            1.772837  0.100722   

                                                                      \
                                  dH_bulk  volume_pa bulk_oxid_state   
                                                                       
compenv slab_id     active_site                                        
nersc   buvivore_13 38.0        -0.568450  12.531646               4   
        dakoputu_58 74.0        -0.735138  11.952412               4   
                    75.0        -0.735138  11.952412               4   
                    76.0        -0.735138  11.952412               4   
                    77.0        -0.735138  11.952412               4   
...                                   ...        ...             ...   
slac    wonataro_02 64.0        -0.599425  15.036089               6   
        wowukoli_59 27.0        -0.691297  10.810322               4   
        wufulafe_03 56.0        -0.700424  11.389867               4   
                    57.0        -0.700424  11.389867               4   
                    58.0        -0.700424  11.389867               4   

                                                    
                                effective_ox_state  
                                                    
compenv slab_id     active_site                     
nersc   buvivore_13 38.0                  5.666667  
        dakoputu_58 74.0                  8.333333  
                    75.0                  7.333333  
                    76.0                  8.333333  
                    77.0                  8.333333  
...                                            ...  
slac    wonataro_02 64.0                  8.000000  
        wowukoli_59 27.0    

In [32]:
kdict = [
    {
        "type": "gaussian",
        "dimension": "single",
        "width": 1.8,
        "scaling": 0.5,
        "scaling_bounds": ((0.0001, 10.),),
        }
    ]

GP_R = GP_Regression(
    kernel_list=kdict,
    regularization=0.01,
    optimize_hyperparameters=True,
    scale_data=False,
    )

In [33]:
data_dict_list = []
num_feat_cols = df_data.features.shape[1]
for num_pca_i in range(1, num_feat_cols + 1, do_every_nth_pca_comp):

    if verbose:
        print("")
        print(40 * "*")
        print(num_pca_i)

    MA = ModelAgent(
        df_features_targets=df_data,
        Regression=GP_R,
        Regression_class=GP_Regression,
        use_pca=True,
        num_pca=num_pca_i,
        adsorbates=adsorbates,
        stand_targets=False,  # True was giving much worse errors, keep False
        )

    MA.run_kfold_cv_workflow(
        k_fold_partition_size=k_fold_partition_size,
        )

    if verbose:
        print("MAE:", np.round(MA.mae, 4))
        print("MA.r2:", np.round(MA.r2, 4))
        print("MAE (in_fold):", np.round(MA.mae_infold, 4))

    data_dict_i = dict()
    data_dict_i["num_pca"] = num_pca_i
    data_dict_i["MAE"] = MA.mae
    data_dict_i["ModelAgent"] = MA
    data_dict_list.append(data_dict_i)

df_models = pd.DataFrame(data_dict_list)
df_models = df_models.set_index("num_pca")




# #########################################################
# Finding best performing model
row_models_i = df_models.sort_values("MAE").iloc[0]

MA_best = row_models_i.ModelAgent

print(4 * "\n")
if verbose:
    print(
        row_models_i.name,
        " PCA components are ideal with an MAE of ",
        np.round(
        row_models_i.MAE,
            4),
        sep="")


****************************************
1
MAE: 0.3512
MA.r2: 0.2501
MAE (in_fold): 0.3972

****************************************
2
MAE: 0.2381
MA.r2: 0.6049
MAE (in_fold): 0.3891

****************************************
3
MAE: 0.193
MA.r2: 0.738
MAE (in_fold): 0.1961

****************************************
4
MAE: 0.1896
MA.r2: 0.7496
MAE (in_fold): 0.1828

****************************************
5
MAE: 0.1807
MA.r2: 0.766
MAE (in_fold): 0.1722

****************************************
6
MAE: 0.1788
MA.r2: 0.7669
MAE (in_fold): 0.1637





6 PCA components are ideal with an MAE of 0.1788


In [34]:
from methods_models import ModelAgent_Plotter

MA_Plot = ModelAgent_Plotter(
    ModelAgent=MA_best,
    layout_shared=layout_shared,
    )

MA_Plot.plot_residuals()
MA_Plot.plot_parity()
MA_Plot.plot_parity_infold()

# # Uncomment to run pca analysis on in-fold regression
# MA.run_pca_analysis()

In [35]:
fig = MA_Plot.plot_residuals__PLT
if show_plot:
    fig.show()

In [36]:
fig = MA_Plot.plot_parity__PLT
if show_plot:
    fig.show()

In [37]:
fig = MA_Plot.plot_parity_infold__PLT
if show_plot:
    fig.show()

In [38]:
from methods_models import plot_mae_vs_pca
plot_mae_vs_pca(
    df_models=df_models,
    layout_shared=layout_shared,
    scatter_marker_props=scatter_marker_props,
    )

### Save Data

In [39]:
# Deleting cinv matrix of GP model to save disk space

for num_pca, row_i in df_models.iterrows():
    MA = row_i.ModelAgent
    MA.cleanup_for_pickle()

In [40]:
data_dict_out = {
    "df_models": df_models,
    "ModelAgent_Plot": MA_Plot,
    }

In [41]:
assert False

AssertionError: 

In [ ]:
# Pickling data ###########################################
directory = os.path.join(root_dir, "out_data")
if not os.path.exists(directory): os.makedirs(directory)
with open(os.path.join(directory, "modelling_data.pickle"), "wb") as fle:
    pickle.dump(data_dict_out, fle)
# #########################################################

In [ ]:
# #########################################################
print(20 * "# # ")
print("All done!")
print("Run time:", np.round((time.time() - ti) / 60, 3), "min")
print("model__mine_GP.ipynb")
print(20 * "# # ")
# #########################################################